In [1]:

import datarobot as dr
import pandas as pd

In [2]:
use_case = dr.Context.use_case
use_case

UseCase(id=66c8a66a6edd694c8ca75f20, name=Subscribers Churn, description=None, models=427, projects=9, datasets=24, notebooks=4, applications=0)

In [17]:
project= dr.Project.get('6744e18dadd76d3a7255f235')

top_model = dr.DatetimeModel.get(model_id='67461c3b7e7a50311055e79d', project=project.id)
top_model

DatetimeModel('eXtreme Gradient Boosted Trees Classifier with Early Stopping (learning rate =0.02)')

In [18]:
prediction_server = dr.PredictionServer.list()[
    0
]  # Deploy to the first prediction server
prediction_server

PredictionServer(https://gannett-media-corp.dynamic.orm.datarobot.com)

In [19]:
# prediction_threshold = top_model.
prediction_threshold = 0.1211
prediction_threshold

0.1211

In [12]:

deployment = dr.Deployment.create_from_learning_model(
    model_id=top_model.id,
    description=f"Deployment Intention to Stop",
    prediction_threshold=prediction_threshold,
    label="Test scoring",
    default_prediction_server_id=prediction_server.id,
)

deployment.update_drift_tracking_settings(
    target_drift_enabled=True, feature_drift_enabled=True
)


# HTML(
#     f"""
# <div style="text-aligh:center;padding:.75rem;"> 
#     <a href="{deployment.get_uri()}" target="_blank" style="background-color:#5371BF;color:white;padding:.66rem .75rem;border-radius:5px;cursor: pointer;">Open Deployment in DataRobot</a>
# </div>"""
# )

NameError: name 'HTML' is not defined

In [20]:
deployment = dr.Deployment.list()[0]
deployment

Deployment(Test scoring)

In [22]:
creds = dr.Credential.list()[0]
creds

Credential('675084059e6137355c620335', 'bigquery-oauth', 'oauth')

In [25]:
creds.credential_id

'675084059e6137355c620335'

In [28]:

intake_settings = {
    'type': 'bigquery',
    'dataset': 'gannett-datascience.DR_Vitalii',
    'table': 'gannett-datascience.DR_Vitalii.wk_training_current_test2',
    'bucket': 'gannett-data-science', 
    'credential_id': creds.credential_id,
}

output_settings = {
    'type': 'bigquery',
    'dataset': 'gannett-datascience.DR_Vitalii',
    'table': 'gannett-datascience.DR_Vitalii.wk_predictions_test2',
    'bucket': 'gannett-data-science',
    'credential_id': creds.credential_id,
}

job = dr.BatchPredictionJob.score(
    deployment=deployment.id,
    intake_settings=intake_settings,
    output_settings=output_settings,
    include_prediction_status=True,
    passthrough_columns=["idsubscrip", "idsystem", "pubcode", "observation_date", "churned", "stop_intent_date"],
)

print("started scoring...", job)
job.wait_for_completion()

started scoring... BatchPredictionJob(batchPredictions, '67508634c5f65b7421179930', status=INITIALIZING)


AsyncProcessUnsuccessfulError: The job did not complete successfully. Job Data: {'jobSpec': {'batchJobType': 'prediction', 'chunkSize': 'auto', 'csvSettings': {'delimiter': ',', 'quotechar': '"', 'encoding': 'utf-8'}, 'numConcurrent': 16, 'abortOnError': True, 'deploymentId': '674df03f8d8dc22fec9eb8b4', 'modelId': '67461c3b7e7a50311055e79d', 'intakeSettings': {'type': 'bigquery', 'dataset': 'gannett-datascience.DR_Vitalii', 'table': 'gannett-datascience.DR_Vitalii.wk_training_current_test2', 'bucket': 'gannett-data-science', 'credentialId': '675084059e6137355c620335'}, 'outputSettings': {'type': 'bigquery', 'dataset': 'gannett-datascience.DR_Vitalii', 'table': 'gannett-datascience.DR_Vitalii.wk_predictions_test2', 'bucket': 'gannett-data-science', 'credentialId': '675084059e6137355c620335'}, 'redactedFields': ['predictionInstance.datarobotKey'], 'maxExplanations': 0, 'skipDriftTracking': False, 'includeProbabilities': True, 'includePredictionStatus': True, 'predictionWarningEnabled': N

In [0]:
ob Data: {'jobSpec': {'batchJobType': 'prediction', 'chunkSize': 'auto', 'csvSettings': {'delimiter': ',', 'quotechar': '"', 'encoding': 'utf-8'}, 'numConcurrent': 16, 'abortOnError': True, 'deploymentId': '674df03f8d8dc22fec9eb8b4', 'modelId': '67461c3b7e7a50311055e79d', 'intakeSettings': {'type': 'bigquery', 'dataset': 'DR_Vitalii', 'table': 'wk_training_current_test2', 'bucket': 'gannett-data-science', 'credentialId': '66c67f3a2a5c59dcd4d7c646'}, 'outputSettings': {'type': 'bigquery', 'dataset': 'DR_Vitalii', 'table': 'wk_predictions_test2', 'bucket': 'gannett-data-science', 'credentialId': '66c67f3a2a5c59dcd4d7c646'}, 'redactedFields': ['predictionInstance.datarobotKey'], 'maxExplanations': 0, 'skipDriftTracking': False, 'includeProbabilities': True, 'includePredictionStatus': True, 'predictionWarningEnabled': None, 'includeProbabilitiesClasses': [], 'disableRowLevelErrorHandling': False, 'columnNamesRemapping': None, 'predictionInstance': {'hostName': 'gannett-media-corp.dynamic.orm.datarobot.com', 'sslEnabled': True, 'datarobotKey': '[redacted]'}, 'passthroughColumns': ['idsubscrip', 'idsystem', 'pubcode', 'observation_date', 'churned', 'stop_intent_date']}, 'status': 'ABORTED', 'statusDetails': '[ERROR] Could not configure intake adaptor at 2024-12-04 16:00:16.258000', 'id': '67507c7dc50c0e9df640b193', 'created': '2024-12-04T15:59:57.314000Z', 'createdBy': {'userId': '66ba31afe1d0c7ab971a5f5d', 'username': 'vperetiatko@gannett.com', 'fullName': 'Vitalii Peretiatko'}, 'logs': ['[INFO] Job created by vperetiatko@gannett.com at 2024-12-04 15:59:57.314000', '[ERROR] Could not configure intake adaptor at 2024-12-04 16:00:16.258000'], 'percentageCompleted': 0.0, 'jobIntakeSize': None, 'jobOutputSize': None, 'scoredRows': 0, 'failedRows': 0, 'skippedRows': 0, 'monitoringBatchId': None, 'queuePosition': 0, 'links': {'self': 'https://app.datarobot.com/api/v2/batchPredictions/67507c7dc50c0e9df640b193/'}, 'elapsedTimeSec': 18, 'source': 'api_client_python', 'queued': True, 'previewAvailable': False, 'batchPredictionJobDefinition': None}

HERE

In [59]:
dataset_id = "6750b35f875560ffb65f2ba1"

dataset = dr.Dataset.get(dataset_id)




In [60]:
job = dr.BatchPredictionJob.score(
    deployment.id,
    intake_settings={
        'type': 'dataset',
        'dataset': dataset,
    },
    output_settings={
        'type': 'localFile',
    },
    include_prediction_status=True,
    passthrough_columns=["idsubscrip", "idsystem", "pubcode", "observation_date", "churned", "stop_intent_date"],
)

In [61]:
job.get_status()

{'id': '6750bd9dc50c0e9df640b4db',
 'status': 'INITIALIZING',
 'percentage_completed': 0.0,
 'elapsed_time_sec': 3,
 'links': {'self': 'https://app.datarobot.com/api/v2/batchPredictions/6750bd9dc50c0e9df640b4db/'},
 'job_spec': {'num_concurrent': 16,
  'deployment_id': '674df03f8d8dc22fec9eb8b4',
  'model_id': '67461c3b7e7a50311055e79d',
  'passthrough_columns': ['idsubscrip',
   'idsystem',
   'pubcode',
   'observation_date',
   'churned',
   'stop_intent_date'],
  'max_explanations': 0,
  'intake_settings': {'type': 'dataset',
   'dataset_id': '6750b35f875560ffb65f2ba1',
   'dataset_version_id': '6750b35f875560ffb65f2ba2'},
  'output_settings': {'type': 'localFile'},
  'batch_job_type': 'prediction',
  'chunk_size': 'auto',
  'csv_settings': {'delimiter': ',', 'quotechar': '"', 'encoding': 'utf-8'},
  'abort_on_error': True,
  'redacted_fields': ['predictionInstance.datarobotKey'],
  'skip_drift_tracking': False,
  'include_probabilities': True,
  'include_prediction_status': True,


In [62]:
bytes_data = job.get_result_when_complete(max_wait=900)
from io import BytesIO

result_df = pd.read_csv(BytesIO(bytes_data))


In [63]:
result_df.head()

,churned_1_PREDICTION,churned_0_PREDICTION,churned_PREDICTION,THRESHOLD,POSITIVE_CLASS,DEPLOYMENT_APPROVAL_STATUS,prediction_status,idsubscrip,idsystem,pubcode,observation_date,churned,stop_intent_date
0,0.027620,0.972380,0,0.1211,1,APPROVED,OK,36369251,100800,WF,2024-08-04,0,NaN
1,0.111293,0.888707,0,0.1211,1,APPROVED,OK,25910917,100700,DI,2024-08-04,0,NaN
2,0.044698,0.955302,0,0.1211,1,APPROVED,OK,29542162,109800,CD,2024-08-04,0,NaN
3,0.093347,0.906653,0,0.1211,1,APPROVED,OK,36149230,104000,AN,2024-08-04,0,NaN
4,0.005537,0.994463,0,0.1211,1,APPROVED,OK,21700261,155800,WA,2024-08-04,0,NaN


In [64]:
result_df.describe()

,churned_1_PREDICTION,churned_0_PREDICTION,churned_PREDICTION,THRESHOLD,POSITIVE_CLASS,idsubscrip,idsystem,churned
count,1.601757e+06,1.601757e+06,1.601757e+06,1.601757e+06,1601757.0,1.601757e+06,1.601757e+06,1.601757e+06
mean,4.447308e-02,9.555269e-01,6.963478e-02,1.211000e-01,1.0,6.949255e+07,3.533366e+05,6.836867e-02
std,5.467461e-02,5.467461e-02,2.545306e-01,3.747004e-16,0.0,9.305847e+06,3.255987e+05,2.523776e-01
min,1.854175e-04,1.596409e-02,0.000000e+00,1.211000e-01,1.0,7.950030e+06,1.007000e+05,0.000000e+00
25%,1.254968e-02,9.440971e-01,0.000000e+00,1.211000e-01,1.0,6.739040e+07,1.042000e+05,0.000000e+00
50%,2.662293e-02,9.733771e-01,0.000000e+00,1.211000e-01,1.0,7.404512e+07,1.263000e+05,0.000000e+00
75%,5.590288e-02,9.874503e-01,0.000000e+00,1.211000e-01,1.0,7.552061e+07,7.000040e+05,0.000000e+00
max,9.840359e-01,9.998146e-01,1.000000e+00,1.211000e-01,1.0,7.624286e+07,8.872000e+05,1.000000e+00


In [65]:
pred_dataset = dr.Dataset.create_from_in_memory_data(
    result_df, 
    fname=f"intention_to_stop_predictions_mid{top_model.id}_pp{result_df.observation_date.max()}",
    use_cases=use_case
)

In [66]:
pred_dataset

Dataset(name='intention_to_stop_predictions_mid67461c3b7e7a50311055e79d_pp2024-08-04', id='6750c04ba26a792e4929df9a')

In [0]:
# import json
# from pathlib import Path

# json_credential = json.loads(
#     Path("PATH TO YOUR JSON SERVICE CREDENTIAL").read_text()
# )  # You can obtain your service credentials in a number of ways

# # google_cloud_credential = dr.Credential.create_gcp(name='GCP Key Credential Test', gcp_key=json_credential, description="For GCP Batch Access")

# job = dr.BatchPredictionJob.score(
#     deployment.id,  # this is the deployment id
#     intake_settings={
#         "type": "bigquery",
#         "dataset": "dr_sample_data",
#         "table": "lending_club",
#         "bucket": "model-staging-dr-demo",  # a bucket is required
#         "credential_id": google_cloud_credential.credential_id,
#     },
#     output_settings={
#         "type": "bigquery",
#         "dataset": "dr_sample_data",
#         "table": "lending_club_predictions",
#         "bucket": "model-staging-dr-demo",  # a bucket is required
#         "credential_id": google_cloud_credential.credential_id,
#     },
# )

# job.get_result_when_complete()